# P2PKH UTXO and send to P2SH(P2WSH(P2PK))

In [1]:
from bitcoinutils.keys import P2shAddress, PrivateKey
from bitcoinutils.script import Script
from bitcoinutils.setup import setup
from bitcoinutils.hdwallet import HDWallet
from bitcoinutils.transactions import Transaction, TxInput, TxOutput
from bitcoinutils.setup import setup
from bitcoinutils.keys import P2shAddress, PublicKey
from bitcoinutils.script import Script
from bitcoinutils.utils import to_satoshis
from bitcoinutils.transactions import Transaction, TxInput, TxOutput, TxWitnessInput
from bitcoinutils.keys import P2pkhAddress, P2shAddress, P2wshAddress, PrivateKey, PublicKey, Address
from bitcoinutils.proxy import NodeProxy
import hashlib
from enum import Enum
from bitcoinutils.setup import setup
from binascii import unhexlify, hexlify
from bitcoinutils.keys import P2shAddress, PrivateKey
from bitcoinutils.script import Script
from bitcoinutils.setup import setup
from bitcoinutils.hdwallet import HDWallet
from bitcoinutils.transactions import Transaction, TxInput, TxOutput, TxWitnessInput
setup("regtest")

'regtest'

In [2]:
# private_key1 = PrivateKey("cTicJXw3ywZptXupB453nmPCznnueXygw8vw1ckyAkQ63RZc7d34")
# private_key2 = PrivateKey("cRQrsYJo8MPPkevELzJ85b3ei3M7p1doFfWv6oMQeFGupxFrq1sv")

In [3]:
from_priv = PrivateKey("cTicJXw3ywZptXupB453nmPCznnueXygw8vw1ckyAkQ63RZc7d34")
from_pub = from_priv.get_public_key()
from_addr = from_pub.get_address()
print("From address:", from_addr.to_string())

From address: mgkcEnxka8PUv8arimkW1S8icS2RD3TG3o


In [4]:
to_priv = PrivateKey("cRQrsYJo8MPPkevELzJ85b3ei3M7p1doFfWv6oMQeFGupxFrq1sv")
to_pub = to_priv.get_public_key()

witness_script = Script([to_pub.to_hex(), "OP_CHECKSIG"])
p2sh_redeem_script = witness_script.to_p2wsh_script_pub_key()  # maybe to_p2sh_...

p2sh_address = P2shAddress.from_script(p2sh_redeem_script)
print("To address:", p2sh_address.to_string())

To address: 2NCNxo7iGGP5hi6YNFudL5QfrP5BBthpn5M


In [5]:
proxy = NodeProxy('deep', 'mdsSgY3ZUXe6x5JIQ81gpNwPPDu-arnaRGh-AGmi0IM').get_proxy()
p2pkh_addr = from_addr
n_blocks = proxy.getblockcount()
n_mempool = proxy.getmempoolinfo()["size"]
total_txs = sum(
    [proxy.getblockstats(block, ["txs"])["txs"] for block in range(1, n_blocks)]
)
all_txs = proxy.listtransactions("*", total_txs + n_mempool, 0, True)
p2sh_txs = [*filter(lambda tx: tx["address"] == p2sh_address.to_string(), all_txs)]
print(p2sh_txs)

[{'address': '2NCNxo7iGGP5hi6YNFudL5QfrP5BBthpn5M', 'category': 'send', 'amount': Decimal('-0.10000000'), 'vout': 1, 'fee': Decimal('-0.00001430'), 'confirmations': 0, 'trusted': True, 'txid': '7a5630126294c0cac59f53ba31dd902591f8791b11f9f0f9db0a20b410eb7f0f', 'wtxid': '39db4ca1cb9a7dc1912ba7174ee164c95da1f767e475ebe5a5544ec9395c37fe', 'walletconflicts': [], 'time': 1694467590, 'timereceived': 1694467590, 'bip125-replaceable': 'yes', 'abandoned': False}]


In [6]:
p2sh_utxo = []
amount_to_send = 0.0
MIN_CONF = 0

for tx in p2sh_txs:
    if not proxy.gettxout(tx["txid"], tx["vout"]):
        print("tx [{}] already spent".format(tx["txid"]))
        continue
    if int(tx["confirmations"]) >= MIN_CONF and tx["category"] == "send":
        tx_in = TxInput(tx["txid"], tx["vout"])
        print(
            "*Found UTXO with {0:.8f} tBTC, TXID : [{1}]".format(
                abs(tx["amount"]), tx["txid"]
            )
        )
        p2sh_utxo += [tx_in]
        amount_to_send += abs(float(tx["amount"]))

if not p2sh_utxo:
    raise RuntimeError("No UTXO's currently associated with that P2SH address")

PRICE_PER_BYTE = 0.00078 / 1024
tx_fee = (len(p2sh_utxo) * 148 + 1 * 34 + 10) * PRICE_PER_BYTE

print("Total spendable amount aggregated from {0} UTXO(s) associated with P2SH address: {1:.8f} tBTC".format(len(p2sh_utxo), amount_to_send))
print("TX fee in satoshis: {0} (Fee Rate is {1:.8f} tBTC/KB)".format(to_satoshis(tx_fee), 0.00078))
amount_to_send -= tx_fee

print("Sending {0} satoshis ({1:.8f} BTC) to P2PKH address [{2}]".format(to_satoshis(amount_to_send), amount_to_send, p2pkh_addr.to_string()))

*Found UTXO with 0.10000000 tBTC, TXID : [7a5630126294c0cac59f53ba31dd902591f8791b11f9f0f9db0a20b410eb7f0f]
Total spendable amount aggregated from 1 UTXO(s) associated with P2SH address: 0.10000000 tBTC
TX fee in satoshis: 14625 (Fee Rate is 0.00078000 tBTC/KB)
Sending 9985375 satoshis (0.09985375 BTC) to P2PKH address [mgkcEnxka8PUv8arimkW1S8icS2RD3TG3o]


In [7]:
p2sh_utxo

[{'txid': '7a5630126294c0cac59f53ba31dd902591f8791b11f9f0f9db0a20b410eb7f0f', 'txout_index': 1, 'script_sig': []}]

In [8]:
# txout = TxOutput(to_satoshis(amount_to_send), p2pkh_addr.to_script_pub_key())
# tx = Transaction(inputs=p2sh_utxo, outputs=[txout], has_segwit=True)
# print("\nRaw transaction:\n" + tx.serialize())

In [9]:
# Generated Transaction Id with the command
# ./bitcoin-cli -regtest sendtoaddress "2NCNxo7iGGP5hi6YNFudL5QfrP5BBthpn5M" 0.1
txid = "7a5630126294c0cac59f53ba31dd902591f8791b11f9f0f9db0a20b410eb7f0f"
vout = 0

txin = TxInput(txid, vout)
txout = TxOutput(5000, p2sh_redeem_script.to_p2sh_script_pub_key())
txout_change = TxOutput(1530000, from_addr.to_script_pub_key())
tx = Transaction([txin], [txout, txout_change])

In [10]:
sig = from_priv.sign_input(tx, 0, from_addr.to_script_pub_key())
txin.script_sig = Script([sig, from_pub.to_hex()])
signed_tx = tx.serialize()
print(signed_tx)

02000000010f7feb10b4200adbf9f0f9111b79f8912590dd31ba539fc5cac094621230567a000000006a47304402202f454be9831c9be09d996a484a5f58744e83aca7b4873cf29ffc43983b7f7dbd022078a592d03bfe17b928be671bd877ae71ce1db3fea8b226cd5fff62a11de6b48d01210283aacf4507f578596fc8d65f393ecf8bc38a9740035c267bf745f89b4497626bffffffff02881300000000000017a914d1e10915ee95cc4e22c0158336f5a3d5777768388790581700000000001976a9140d8d3508655bdf3bada0c33a87110f4eed366abc88ac00000000


In [11]:
response_status = proxy.testmempoolaccept([signed_tx])[0]

In [12]:
response_status

{'txid': '4b83be5a725454adb6b8def8a38d39a93b9a18473be8b217e163150e4a3aa8a5',
 'wtxid': '4b83be5a725454adb6b8def8a38d39a93b9a18473be8b217e163150e4a3aa8a5',
 'allowed': False,
 'reject-reason': 'mandatory-script-verify-flag-failed (Script evaluated without error but finished with a false/empty top stack element)'}

# P2SH(P2WSH(P2PK)) UTXO to a P2PKH UTXO

In [1]:
from bitcoinutils.keys import P2shAddress, PrivateKey
from bitcoinutils.script import Script
from bitcoinutils.setup import setup
from bitcoinutils.hdwallet import HDWallet
from bitcoinutils.transactions import Transaction, TxInput, TxOutput
from bitcoinutils.setup import setup
from bitcoinutils.keys import P2shAddress, PublicKey
from bitcoinutils.script import Script
from bitcoinutils.utils import to_satoshis
from bitcoinutils.transactions import Transaction, TxInput, TxOutput, TxWitnessInput
from bitcoinutils.keys import P2pkhAddress, P2shAddress, P2wshAddress, PrivateKey, PublicKey, Address
from bitcoinutils.proxy import NodeProxy
import hashlib
from enum import Enum
from bitcoinutils.setup import setup
from binascii import unhexlify, hexlify
from bitcoinutils.keys import P2shAddress, PrivateKey
from bitcoinutils.script import Script
from bitcoinutils.setup import setup
from bitcoinutils.hdwallet import HDWallet
from bitcoinutils.transactions import Transaction, TxInput, TxOutput, TxWitnessInput
setup("regtest")

'regtest'

In [2]:
from_priv = PrivateKey("cTicJXw3ywZptXupB453nmPCznnueXygw8vw1ckyAkQ63RZc7d34")
print(from_priv.to_wif())
from_pub = from_priv.get_public_key()
from_addr = from_pub.get_address()

witness_script = Script([from_pub.to_hex(), "OP_CHECKSIG"])
p2sh_redeem_script = witness_script.to_p2wsh_script_pub_key()
print("From address:", P2shAddress.from_script(p2sh_redeem_script).to_string())

cTicJXw3ywZptXupB453nmPCznnueXygw8vw1ckyAkQ63RZc7d34
From address: 2N7y1bV4YbcsYNyHJrRwZdGBYPmeLUt1xnM


In [3]:
to_priv = PrivateKey("cRQrsYJo8MPPkevELzJ85b3ei3M7p1doFfWv6oMQeFGupxFrq1sv")
to_address = to_priv.get_public_key().get_address()
print("To address:", to_address.to_string())

To address: mhZ7PvZZp5dafdGdds37m5AbZzArtqFsgT


In [4]:
# Generated Transaction Id with the command
# ./bitcoin-cli -regtest sendtoaddress "mhZ7PvZZp5dafdGdds37m5AbZzArtqFsgT" 0.1
txid = "5cf932c5c391cffbf1d92d13221900a38c4250217ce80d3e9371b9197b623a23"
vout = 0
amount = 5000

txin = TxInput(txid, vout)
txout = TxOutput(3000, to_address.to_script_pub_key())
tx = Transaction([txin], [txout], has_segwit=True)

sig = from_priv.sign_segwit_input(tx, 0, witness_script, amount)

txin.script_sig = Script([p2sh_redeem_script.to_hex()])
tx.witnesses.append(TxWitnessInput([sig, witness_script.to_hex()]))

signed_tx = tx.serialize()

print(signed_tx)

02000000000101233a627b19b971933e0de87c2150428ca3001922132dd9f1fbcf91c3c532f95c0000000023220020b93a2e59a5c28fcbb4f2832b1ffd5808024c8902c1f8d06b63495adb06bc7820ffffffff01b80b0000000000001976a9141658ae48e77ce9b2f30f8949af51eca5328155b488ac024730440220736f51d34cb61e4b14f4f9845b55d601b922cca9351314bae53d4eb5b849a9c0022052cd5dd575248cb9c87a73e0e11433f210639857605e650149571346efa3e8160123210283aacf4507f578596fc8d65f393ecf8bc38a9740035c267bf745f89b4497626bac00000000


In [5]:
proxy = NodeProxy('deep', 'mdsSgY3ZUXe6x5JIQ81gpNwPPDu-arnaRGh-AGmi0IM').get_proxy()
response_status = proxy.testmempoolaccept([signed_tx])[0]
response_status

{'txid': 'e70ce6c653551161ebcefc2d64813e8bf79c974925640b7dc33f7d67f2cf7947',
 'wtxid': 'e305228319d62935289aad33500223263805ca789f574d0d2c4448c45c316497',
 'allowed': False,
 'reject-reason': 'bad-witness-nonstandard'}